1. NumPy와 메모리 공유 (from_numpy)

In [1]:
import torch
import numpy as np

# 1-1. NumPy 배열 생성
numpy_arr = np.array([1,2,3])
print(f"원본 NumPy 배열: {numpy_arr}")

# 1-2. from_numpy로 텐서 생성 (메모리 공유)
tensor_shared = torch.from_numpy(numpy_arr)
print(f"공유 텐서 (from_numpy): {tensor_shared}")

# 1-3. (비교) tensor()로 텐서 생성 (메모리 복사)
tensor_copied = torch.tensor(numpy_arr)
print(f"복사 텐서 (tensor): {tensor_copied}")

# --- 메모리 공유 테스트 ---
print("\n" + "-"*20)
print("NumPy 배열의 0번 값을 99로 변경합니다.")
numpy_arr[0] = 99

print(f"NumPy 배열: {numpy_arr}")
print(f"공유 텐서 (from_numpy): {tensor_shared} <-- 같이 바뀜!")
print(f"복사 텐서 (tensor): {tensor_copied} <-- 안 바뀜!")
print("-" * 20 + "\n")

원본 NumPy 배열: [1 2 3]
공유 텐서 (from_numpy): tensor([1, 2, 3])
복사 텐서 (tensor): tensor([1, 2, 3])

--------------------
NumPy 배열의 0번 값을 99로 변경합니다.
NumPy 배열: [99  2  3]
공유 텐서 (from_numpy): tensor([99,  2,  3]) <-- 같이 바뀜!
복사 텐서 (tensor): tensor([1, 2, 3]) <-- 안 바뀜!
--------------------



3일차 코드를 GPU에서 실행하기

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# --- 0. 장치 설정 (오늘의 핵심 1) ---
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"현재 학습에 사용할 장치: {device}")

# --- 1. 데이터 준비 및 GPU로 전송 (오늘의 핵심 2) ---
x_train = torch.tensor([[0.0], [1.0], [2.0], [3.0]], dtype=torch.float32).to(device)
y_train = torch.tensor([[1.0], [3.0], [5.0], [7.0]], dtype=torch.float32).to(device)

print(f"\nx_train이 있는 장치: {x_train.device}")

# --- 2. 모델, 손실함수, 옵티마이저 정의 ---

# 모델을 생성
model = nn.Linear(1, 1)
# 모델을 GPU로 전송 (오늘의 핵심 3)
model.to(device)

# 손실 함수와 옵티마이저 정의 (이것들은 GPU로 보낼 필요 없음)
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# --- 3. 학습 루프 ---
epochs = 1000

for epoch in range(epochs + 1):

    # 1. 순전파: model과 x_train이 모두 'device'에 있으므로 연산 가능
    prediction = model(x_train)

    # 2. 손실 계산: prediction과 y_train이 모두 'device'에 있으므로 연산 가능
    loss = loss_fn(prediction, y_train)

    # 3. 기울기 초기화
    optimizer.zero_grad()

    # 4. 역전파: loss가 'device'에 있으므로 .backward()도 GPU에서 수행됨
    loss.backward()

    # 5. 파라미터 업데이트: model.parameters()가 'device'에 있으므로 GPU에서 업데이트됨
    optimizer.step()

    if epoch % 100 == 0:
        # .item()은 GPU/CPU 텐서에서 숫자 값만 빼올 때 사용
        print(f"Epoch {epoch:4d}/{epochs} | Loss: {loss.item():.4f}")

# --- 4. 예측 (예측할 데이터도 GPU로 보내야 함) ---
# .to(device)를 잊지 마세요!
test_x = torch.tensor([[4.0]]).to(device)
predicted_y = model(test_x)

# .item()으로 최종 결과(숫자)만 가져옴
print(f"\n4시간 공부했을 때 예상 성적: {predicted_y.item():.3f} 점")

현재 학습에 사용할 장치: cpu

x_train이 있는 장치: cpu
Epoch    0/1000 | Loss: 17.0379
Epoch  100/1000 | Loss: 0.0432
Epoch  200/1000 | Loss: 0.0131
Epoch  300/1000 | Loss: 0.0040
Epoch  400/1000 | Loss: 0.0012
Epoch  500/1000 | Loss: 0.0004
Epoch  600/1000 | Loss: 0.0001
Epoch  700/1000 | Loss: 0.0000
Epoch  800/1000 | Loss: 0.0000
Epoch  900/1000 | Loss: 0.0000
Epoch 1000/1000 | Loss: 0.0000

4시간 공부했을 때 예상 성적: 8.999 점
